# Determining the salary for player in FIFA 18

This dataset obtained from www.kaggle.com contains 185 fields for every player in FIFA 18:
 - Player info such as age, club, league, nationality, salary and physical attributes
 - All playing attributes, such as finishing and dribbling
 - Special attributes like skill moves and international reputation
 - Traits and specialities
 - Overall, potential, and ratings for each position
 
We would construct a regression model trying to predict the salary of each player. The idea covered in the textbook but not covered well in the lectures that we use is: an ensemble of different models: polynomial regression, decision tree, random forest, (K-nearest).

Some of the columns contain special characters so we use UTF-8 encoding to read the .csv in properly. 

# Import and explore data

In [2]:
import pandas as pd
pd.options.display.max_columns = 50
df = pd.read_csv("complete.csv", encoding="utf-8")
df.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,weight_kg,body_type,real_face,flag,nationality,photo,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,...,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera División,1985-02-05,185.0,80.0,C. Ronaldo,True,https://cdn.sofifa.org/flags/38@3x.png,Portugal,https://cdn.sofifa.org/18/players/20801.png,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,...,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera División,1987-06-24,170.0,72.0,Messi,True,https://cdn.sofifa.org/flags/52@3x.png,Argentina,https://cdn.sofifa.org/18/players/158023.png,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,...,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,68.0,Neymar,True,https://cdn.sofifa.org/flags/54@3x.png,Brazil,https://cdn.sofifa.org/18/players/190871.png,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,176580,L. Suárez,Luis Suárez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera División,1987-01-24,182.0,86.0,Normal,True,https://cdn.sofifa.org/flags/60@3x.png,Uruguay,https://cdn.sofifa.org/18/players/176580.png,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,...,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,92.0,Normal,True,https://cdn.sofifa.org/flags/21@3x.png,Germany,https://cdn.sofifa.org/18/players/167495.png,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


# Observe
- The data has a lot of very detail features such as international reputation, work rate at attacking and defending, etc. 
- We drop columns that we think do not correlate with salary: ID, special, club_logo, flag, photo.
- As goal keepers (gk) have a separate set of properties, we move them to the gk dataframe. The other player data move to data dataframe.
- We drop the rows with null salary. 
(- Cristiano Ronaldo, L.Messi and L. Suárez have exceptional high salary because they have several uncountable value. Therefore, I will drop these 3.)
- Change work_rate from scale Low, Medium, High to scale 1, 2, 3
- Change boolean to scale 0, 1
- In player dataframe, drop all goalkeeper skills.


In [3]:
# Drop exceptions
# df.drop(['20801', '158023', '176580'], axis=0)

# Drop columns.
for i in {'ID','special','club_logo','flag','photo','real_face', 'prefers_gk'}:
    del df[i]

# Drop zero salary row.
df = df[df['eur_wage']!=0]

df.head(5)

,name,full_name,club,age,league,birth_date,height_cm,weight_kg,body_type,nationality,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,...,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb
0,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,32,Spanish Primera División,1985-02-05,185.0,80.0,C. Ronaldo,Portugal,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,33,80,5,5,4,High,...,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,L. Messi,Lionel Messi,FC Barcelona,30,Spanish Primera División,1987-06-24,170.0,72.0,Messi,Argentina,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,26,61,5,4,4,Medium,...,True,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,25,French Ligue 1,1992-02-05,175.0,68.0,Neymar,Brazil,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,30,60,5,5,5,High,...,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,L. Suárez,Luis Suárez,FC Barcelona,30,Spanish Primera División,1987-01-24,182.0,86.0,Normal,Uruguay,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,42,81,5,4,4,High,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,M. Neuer,Manuel Neuer,FC Bayern Munich,31,German Bundesliga,1986-03-27,193.0,92.0,Normal,Germany,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,60,91,5,1,4,Medium,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [4]:
df['work_rate_att'].replace({'High':3,'Medium':2,'Low':1},inplace = True)
df['work_rate_def'].replace({'High':3,'Medium':2,'Low':1},inplace = True)

In [6]:
start_column = df.columns.get_loc("1_on_1_rush_trait")
end_column = df.columns.get_loc("prefers_lcb") + 1
for i in range (start_column,end_column):
    a = df.iloc[:,i]
    a.replace({True:1, False:0}, inplace = True)
df.head(5)
df.corr()

,age,height_cm,weight_kg,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,work_rate_def,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,...,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb
age,1.000000,0.076281,0.219359,0.084194,0.155772,0.065595,0.461687,-0.223096,-0.134023,0.249968,0.340831,0.181138,0.226417,0.421913,0.238032,-0.029634,0.059021,-0.007710,0.077941,0.138184,0.067433,0.145631,0.133655,0.135602,0.006738,...,-0.060713,NaN,NaN,NaN,-0.022845,NaN,NaN,0.020532,0.001975,-0.027455,-0.050805,0.015812,-0.013770,-0.011628,-0.013416,0.066488,0.054298,0.014879,0.000918,NaN,NaN,NaN,NaN,NaN,NaN
height_cm,0.076281,1.000000,0.766964,0.002254,0.020513,0.001133,0.042412,0.003693,-0.386525,-0.073631,-0.193049,-0.302016,0.112747,0.407655,0.042985,-0.445009,-0.186258,-0.241104,0.080242,-0.490013,-0.365572,0.008259,-0.368053,-0.353041,-0.499407,...,-0.127575,NaN,NaN,NaN,-0.252322,NaN,NaN,-0.094539,-0.042268,0.005600,-0.137952,-0.068199,-0.228089,-0.167392,-0.255061,-0.046457,0.329832,-0.100440,-0.048423,NaN,NaN,NaN,NaN,NaN,NaN
weight_kg,0.219359,0.766964,1.000000,0.037482,0.060079,0.031917,0.136853,-0.012786,-0.331960,0.008019,-0.106589,-0.213173,0.119406,0.484860,0.084507,-0.391453,-0.150374,-0.201001,0.078994,-0.405585,-0.297591,0.021724,-0.310616,-0.275641,-0.430399,...,-0.125631,NaN,NaN,NaN,-0.220727,NaN,NaN,-0.080868,-0.032242,0.030919,-0.132501,-0.054908,-0.206729,-0.166409,-0.216677,-0.040444,0.274909,-0.087638,-0.035032,NaN,NaN,NaN,NaN,NaN,NaN
eur_value,0.084194,0.002254,0.037482,1.000000,0.851180,0.994149,0.629889,0.587715,0.213986,0.329352,0.414667,0.412385,0.121695,0.252688,0.720930,0.232526,0.162281,0.138517,0.044432,0.248176,0.256932,0.189810,0.324497,0.287659,0.268396,...,0.058056,NaN,NaN,NaN,0.025691,NaN,NaN,-0.044272,-0.005929,0.039356,0.074681,0.046645,0.060400,0.041394,0.033947,0.028209,-0.018310,-0.044888,-0.005293,NaN,NaN,NaN,NaN,NaN,NaN
eur_wage,0.155772,0.020513,0.060079,0.851180,1.000000,0.852157,0.599680,0.519599,0.172405,0.299968,0.389225,0.374403,0.154792,0.270477,0.704308,0.197292,0.141900,0.119981,0.058956,0.243171,0.233694,0.208131,0.308861,0.268737,0.249284,...,0.047318,NaN,NaN,NaN,0.011274,NaN,NaN,-0.017760,0.012360,0.035163,0.055313,0.030441,0.029976,0.030773,0.020343,0.031422,-0.002992,-0.017111,0.010723,NaN,NaN,NaN,NaN,NaN,NaN
eur_release_clause,0.065595,0.001133,0.031917,0.994149,0.852157,1.000000,0.611823,0.593943,0.210488,0.318600,0.404676,0.404316,0.124241,0.244415,0.710990,0.225733,0.156370,0.136401,0.046168,0.240770,0.249012,0.183494,0.316943,0.274634,0.262194,...,0.059683,NaN,NaN,NaN,0.024284,NaN,NaN,-0.042075,-0.005947,0.036352,0.075073,0.043399,0.055535,0.040833,0.031762,0.027383,-0.017198,-0.041553,-0.002288,NaN,NaN,NaN,NaN,NaN,NaN
overall,0.461687,0.042412,0.136853,0.629889,0.599680,0.611823,1.000000,0.678303,0.259583,0.461959,0.644329,0.588724,0.320891,0.558980,0.527626,0.225683,0.201536,0.149500,0.082098,0.390586,0.321400,0.341189,0.488542,0.379301,0.356534,...,0.024629,NaN,NaN,NaN,0.044737,NaN,NaN,-0.001250,0.007865,0.007667,0.039823,0.043774,0.073374,0.029920,0.066049,0.085534,0.037021,-0.005006,0.012849,NaN,NaN,NaN,NaN,NaN,NaN
potential,-0.223096,0.003693,-0.012786,0.587715,0.519599,0.593943,0.678303,1.000000,0.303379,0.294241,0.415479,0.463291,0.158637,0.233354,0.423505,0.219561,0.159521,0.130200,0.031046,0.249513,0.241669,0.206602,0.368308,0.254910,0.313999,...,0.064242,NaN,NaN,NaN,0.026141,NaN,NaN,-0.036059,-0.001977,0.016267,0.068616,0.026103,0.087872,0.047620,0.041862,0.039369,0.006538,-0.040696,0.002834,NaN,NaN,NaN,NaN,NaN,NaN
pac,-0.134023,-0.386525,-0.331960,0.213986,0.172405,0.210488,0.259583,0.303379,1.000000,0.347289,0.302124,0.56

In [5]:
# Put gk data to gk dataframe.
gk = df[df['gk'].notnull()]

# And drop all the null columns (properties not correlated with a goal keeper salary) in gk.
rs_column = gk.columns.get_loc("rs")
lcb_column = gk.columns.get_loc("lcb") + 1
gk = gk.drop(gk.columns[rs_column:lcb_column],axis=1)

# Finally, we drop the gk column in the df dataframe as not gk players do not have this property.
data = df.drop('gk',axis =1 )
data = df.drop(['gk_diving','gk_handling','gk_kicking','gk_positioning', 'gk_reflexes'],axis =1)
data.dropna()
data.head(5)

,name,full_name,club,age,league,birth_date,height_cm,weight_kg,body_type,nationality,eur_value,eur_wage,eur_release_clause,overall,potential,pac,sho,pas,dri,def,phy,international_reputation,skill_moves,weak_foot,work_rate_att,...,prefers_rw,prefers_rf,prefers_ram,prefers_rcm,prefers_rm,prefers_rdm,prefers_rcb,prefers_rb,prefers_rwb,prefers_st,prefers_lw,prefers_cf,prefers_cam,prefers_cm,prefers_lm,prefers_cdm,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb
0,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,32,Spanish Primera División,1985-02-05,185.0,80.0,C. Ronaldo,Portugal,95500000.0,565000.0,195800000.0,94,94,90,93,82,90,33,80,5,5,4,3,...,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,L. Messi,Lionel Messi,FC Barcelona,30,Spanish Primera División,1987-06-24,170.0,72.0,Messi,Argentina,105000000.0,565000.0,215300000.0,93,93,89,90,86,96,26,61,5,4,4,2,...,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,25,French Ligue 1,1992-02-05,175.0,68.0,Neymar,Brazil,123000000.0,280000.0,236800000.0,92,94,92,84,79,95,30,60,5,5,5,3,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,L. Suárez,Luis Suárez,FC Barcelona,30,Spanish Primera División,1987-01-24,182.0,86.0,Normal,Uruguay,97000000.0,510000.0,198900000.0,92,92,82,90,79,87,42,81,5,4,4,3,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M. Neuer,Manuel Neuer,FC Bayern Munich,31,German Bundesliga,1986-03-27,193.0,92.0,Normal,Germany,61000000.0,230000.0,100700000.0,92,92,91,90,95,89,60,91,5,1,4,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#Analyze
